In [1]:
import pandas as pd

In [2]:
def segment_data(df,year):
    pos_dict = {}
    pos = ['PG','SG','PF','SF','C','F','G']
    
    for p in pos:
        pos_dict[p] = df.loc[df['POS'] == p]
        pos_dict[p]["Year"] = year
        
    return pos_dict

In [3]:
def get_data():
    from os import walk
    
    regular = {}
    post = {}
    
    dirs = list(walk("./player_stats/"))[0][1]
    for i in dirs:
        year = int(i)
        regular[year] = segment_data(pd.read_csv("./player_stats/"+i+"/regular_season.csv"),year)
        post[year] = segment_data(pd.read_csv("./player_stats/"+i+"/post_season.csv"),year)
        
    seasons = {"regular":regular,"post":post}
    return seasons

In [4]:
def player_slider():
    slider = dcc.Slider(
                        id = 'n_players',
                        min = 1,
                        max = 10,
                        value = 5,
                        marks = {str(n): str(n) for n in range(1,11)},
                        step = None,
                        tooltip={'always_visible':False}
                    )
    return slider

In [5]:
def year_slider():
    slider = dcc.RangeSlider(
                        id = 'years',
                        min = min(seasons["regular"].keys()),
                        max = max(seasons["regular"].keys()),
#                         value = max(reg_season.keys()),
                        marks = {year: str(year) for year in seasons["regular"].keys()},
#                         step = None,
                        value=[min(seasons["regular"].keys()), max(seasons["regular"].keys())]
#                         tooltip={'always_visible':False}
                    )
    return slider

In [6]:
def player_stats():
    stats = dcc.RadioItems(
                            options=[
                                {'label': 'PTS','value':'PTS'},
                                {'label': 'AST','value':'AST'},
                                {'label': 'REB','value':'REB'},
                                {'label': 'BLK','value':'BLK'},
                                {'label': 'STL','value':'STL'}
                            ],
                            value = 'PTS',
                            labelStyle={'display': 'inline-block'},
                            id = "stats",
                            style = {'color':colours['radio_button'],
                                     'fontSize':text_size['radio_button']}
                    )
    return stats

In [7]:
def pos_dropdown():
    d = dcc.Dropdown(
                    options = [
                        {"value":"PG", "label":"Point Guard"},
                        {"value":"SG", "label":"Shooting Guard"},
                        {"value":"G", "label":"Guard"},
                        {"value":"PF", "label":"Power Forward"},
                        {"value":"SF", "label":"Small Forward"},
                        {"value":"F", "label":"Forward"},
                        {"value":"C", "label":"Centre"}
                    ],
                    value = "PG",
                    id = "pos_drop"
                )
    return d

In [8]:
def season_dropdown():
    d = dcc.Dropdown(
                    options = [
                        {"value":"regular", "label":"Regular"},
                        {"value":"post", "label":"Post"}
                    ],
                    value = "regular",
                    id = "season_drop"
                )
    return d

In [9]:
from jupyter_dash import JupyterDash
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
from datetime import date
from dash import dcc

season_types = ["regular_season","post_season"]
r_season = season_types[0]
p_season = season_types[1]


y = 2021
pos = 'PG'
top_n_players = 5

seasons = get_data()

sorted_df = seasons["regular"][y][pos].sort_values("PTS",ascending = False).head(top_n_players)

fig = px.bar(sorted_df, x='PTS', y="Name", color="Team", barmode='overlay',opacity=1)
fig.update_layout(title_text="PTS for top "+str(top_n_players)+" players at "+ pos +\
                  " in the "+str(y) +" regular season", title_x=0.5)




app = JupyterDash(__name__)
colours = {'text': '#7FDBFF', 'background':'#333333','radio_button':'#BBBBBB'}
text_size = {'H1':48,'H2':40,'text':28,'radio_button':20}

app.layout = html.Div(style={'backgroundColor':colours['background'],'fontFamily':'Arial'}, children=[

    html.H1(children='NBA Data visualisation',
        style = {'textAlign': 'center',
                 'color':colours['text'],
                 'fontSize':text_size['H1']}),


    html.Div(children=[
        
            html.Div([html.Label("Position: ", style = {'textAlign': 'center',
                         'color':colours['text'],
                         'fontSize':text_size['text']}),
                        pos_dropdown()
                    ],
                style = {'textAlign': 'left',"flex":1}
            ),
        
            html.Div([html.Label("Season: ", style = {'textAlign': 'center',
                         'color':colours['text'],
                         'fontSize':text_size['text']}),
                        season_dropdown()
                     ],
                style = {'textAlign': 'center',"flex":1}
            ),
        
        
            html.Div([html.Label("Stats", style = {'textAlign': 'center',
                         'color':colours['text'],
                         'fontSize':text_size['text']}),
                        player_stats()
                     ],
                style = {'textAlign': 'right',"flex":1}
            )
        
        
            

        ],style = {'display':'flex','flex-direction': 'row'}
    ),
    
    player_slider(),
    
    dcc.Graph(figure = fig, id = 'graph'),
    
    year_slider()
])

@app.callback(
    Output('graph','figure'),
    Input('stats','value'),
    Input('years','value'),
    Input('pos_drop','value'),
    Input('n_players','value'),
    Input('season_drop','value'))
def update_figure(stat,y,pos,top_n_players,season_type):
    
    sorted_df = seasons[season_type][y][pos].sort_values(stat,ascending = False).head(top_n_players)
    fig = px.bar(sorted_df, x=stat, y='Name', color="Team", barmode='overlay', opacity=1)
    
    fig.update_layout(yaxis={'categoryorder':'max ascending'},
                      title_text= stat+ " for top "+str(top_n_players)+" players at "+ pos +\
                      " in the "+str(y) +" regular season", title_x=0.5)

    return fig

/usr/lib/python3/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
/usr/lib/python3/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [10]:
app.run_server(mode = "external")

Dash app running on http://127.0.0.1:8050/


In [75]:
from jupyter_dash import JupyterDash
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
from datetime import date
from dash import dcc

season_types = ["regular_season","post_season"]
r_season = season_types[0]
p_season = season_types[1]


y = 2021
pos = 'PG'
top_n_players = 5

seasons = get_data()

sorted_df = pd.DataFrame()
for k in seasons['regular'].keys():
    d = seasons["regular"][k][pos].sort_values("PTS",ascending = False).head(top_n_players)
    sorted_df = pd.concat([sorted_df,d])

fig = px.scatter(sorted_df, x='Year', y="PTS", color="Name",text="Team")
# fig = px.line(sorted_df, x='Year', y="PTS", color="Name",markers=True)


fig.update_layout(title_text="PTS for top "+str(top_n_players)+" players at "+ pos +\
                  " in the "+str(y) +" regular season", title_x=0.5)


app = JupyterDash(__name__)
colours = {'text': '#7FDBFF', 'background':'#333333','radio_button':'#BBBBBB'}
text_size = {'H1':48,'H2':40,'text':28,'radio_button':20}

app.layout = html.Div(style={'backgroundColor':colours['background'],'fontFamily':'Arial'}, children=[

    html.H1(children='NBA Data visualisation',
        style = {'textAlign': 'center',
                 'color':colours['text'],
                 'fontSize':text_size['H1']}),


    html.Div(children=[
        
            html.Div([html.Label("Position: ", style = {'textAlign': 'center',
                         'color':colours['text'],
                         'fontSize':text_size['text']}),
                        pos_dropdown()
                    ],
                style = {'textAlign': 'left',"flex":1}
            ),
        
            html.Div([html.Label("Season: ", style = {'textAlign': 'center',
                         'color':colours['text'],
                         'fontSize':text_size['text']}),
                        season_dropdown()
                     ],
                style = {'textAlign': 'center',"flex":1}
            ),
        
        
            html.Div([html.Label("Stats", style = {'textAlign': 'center',
                         'color':colours['text'],
                         'fontSize':text_size['text']}),
                        player_stats()
                     ],
                style = {'textAlign': 'right',"flex":1}
            )
        
        
            

        ],style = {'display':'flex','flex-direction': 'row'}
    ),
    
    player_slider(),
    
    dcc.Graph(figure = fig, id = 'graph'),
    
    year_slider()
])

@app.callback(
    Output('graph','figure'),
    Input('stats','value'),
    Input('years','value'),
    Input('pos_drop','value'),
    Input('n_players','value'),
    Input('season_drop','value'))
def update_figure(stat,y,pos,top_n_players,season_type):
    sorted_df = pd.DataFrame()
    
    for k in range(y[0],y[1]+1):
        d = seasons[season_type][k][pos].sort_values("PTS",ascending = False).head(top_n_players)
        sorted_df = pd.concat([sorted_df,d])
    
#     sorted_df = seasons[season_type][y][pos].sort_values(stat,ascending = False).head(top_n_players)
    fig = px.scatter(sorted_df, x='Year', y=stat, color="Name")
    
    fig.update_layout(
        xaxis = dict(
            tickmode = 'array',
            tickvals = list(range(y[0],y[1]+1)),
            ticktext = list(range(y[0],y[1]+1))
        )
    )

    
#     fig.update_layout(yaxis={'categoryorder':'max ascending'},
#                       title_text= stat+ " for top "+str(top_n_players)+" players at "+ pos +\
#                       " in the "+str(y) +" regular season", title_x=0.5)

    return fig

/usr/lib/python3/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [76]:
app.run_server(mode = "inline")

/home/blunt/.local/lib/python3.7/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.



---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
<ipython-input-75-70e3732b987b> in update_figure(
    stat='PTS',
    y=[2001, 2021],
    pos=None,
    top_n_players=2,
    season_type='regular'
)
     92 
     93     for k in range(y[0],y[1]+1):
---> 94         d = seasons[season_type][k][pos].sort_values("PTS",ascending = False).head(top_n_players)
        d = undefined
        global seasons = {'regular': {2013: {'PG':                   Name POS  FGM   FGA   FG%  3PM  3PA   3P%  FTM  FTA  ...   \
2          Rajon Rondo  PG  5.9  12.2  48.4  0.3  1.3  24.0  1.6  2.4  ...    
5        Avery Bradley  PG  3.8   9.6  40.2  0.8  2.5  31.7  0.7  1.0  ...    
21      Deron Williams  PG  6.3  14.4  44.0  2.2  5.7  37.8  4.1  4.7  ...    
25         C.J. Watson  PG  2.4   5.7  41.8  1.1  2.7  41.1  0.9  1.1  ...    
32      Tyshawn Taylor  PG  0.8   2.3  36.8  0.2  0.3  46.2  0.4  0.7  ... 